In [2]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
file = r'C:\Users\hyunj\machin_prj\대중교통위치\전체역_업데이트.xlsx'

df = pd.read_excel(file)
# 서울 중심 좌표 설정
seoul_center = [37.5665, 126.9780]
m = folium.Map(location=seoul_center, zoom_start=11)

# 마커 클러스터 추가
marker_cluster = MarkerCluster().add_to(m)

# 데이터프레임을 순회하며 각 역의 위치에 마커 추가
for _, row in df.iterrows():
    station_name = row['name']
    latitude = row['위도']
    longitude = row['경도']
    line = row['line']
    popup_text = f"{station_name} ({line})"
    folium.Marker(location=[latitude, longitude], popup=popup_text).add_to(marker_cluster)

# 지도 저장
output_path = "seoul_subway_map.html"
m.save(output_path)

output_path


'seoul_subway_map.html'

In [14]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
file = r'C:\Users\hyunj\machin_prj\대중교통위치\국토교통부_전국 버스정류장 위치정보_20241028.csv'

df = pd.read_csv(file, encoding='cp949')

# 서울 중심 좌표 설정
map_center = [37.5665, 126.9780]
m = folium.Map(location=seoul_center, zoom_start=11)
bus_stop_df_clean = df.dropna(subset=['위도', '경도'])

# Folium 지도 다시 생성
m = folium.Map(location=map_center, zoom_start=7)
marker_cluster = MarkerCluster().add_to(m)

# 마커 추가
for _, row in bus_stop_df_clean.iterrows():
    lat = row['위도']
    lon = row['경도']
    name = row['정류장명']
    folium.Marker(
        location=[lat, lon],
        popup=name
    ).add_to(marker_cluster)

# HTML 파일로 저장
output_path = "bus_stops_map.html"
m.save(output_path)

output_path

'bus_stops_map.html'

In [18]:
import pandas as pd
import folium
import json
from shapely.geometry import Point, shape

file = r'C:\Users\hyunj\machin_prj\대중교통위치\전체역_업데이트.xlsx'

# 1. 지하철 엑셀 데이터 불러오기
df = pd.read_excel(file)

# 2. GeoJSON 파일 불러오기 (행정구역 경계)
geojson_path = r'C:\Users\hyunj\machin_prj\대중교통위치\법정구역_시군구_simplified.geojson'
with open(geojson_path, encoding='utf-8') as f:
    geojson = json.load(f)

# 3. 각 자치구의 다각형 및 이름 저장
region_polygons = []
for feature in geojson['features']:
    poly = shape(feature['geometry'])
    name = feature['properties'].get('SIG_KOR_NM')
    region_polygons.append((name, poly))

# 4. 각 지하철역의 위경도를 기반으로 자치구 찾기
def find_region(lat, lng):
    point = Point(lng, lat)
    for name, polygon in region_polygons:
        if polygon.contains(point):
            return name
    return None

# 5. 자치구 이름 열 생성
df['자치구'] = df.apply(lambda row: find_region(row['위도'], row['경도']), axis=1)

# 6. 자치구별 지하철역 수 집계
region_station_counts = df['자치구'].value_counts().reset_index()
region_station_counts.columns = ['SIG_KOR_NM', 'station_count']

# 7. Choropleth 지도 생성
seoul_center = [37.5665, 126.9780]
m = folium.Map(location=seoul_center, zoom_start=11)

folium.Choropleth(
    geo_data=geojson,
    data=region_station_counts,
    columns=['SIG_KOR_NM', 'station_count'],
    key_on='feature.properties.SIG_KOR_NM',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='자치구별 지하철역 수'
).add_to(m)

# 8. 지도 저장
m.save("seoul_station_choropleth_by_region.html")
